### 디렉터리 감시를 통한 보안 위협 탐지

In [1]:
# 디렉터리 내 파일 목록 관리 및 정규식 처리를 위한 모듈
import os
import re
import time

In [4]:
# 감시할 디렉터리 초기 설정 및 초기 파일 목록 기록

# 감시할 디렉터리 경로 지정 (현재 폴더 안의 'monitor_directory' 폴더)
monitor_dir = "./monitor_directory"

# 디렉터리 존재 여부 확인, 없으면 생성
if not os.path.exists(monitor_dir):
    os.makedirs(monitor_dir)
    print(f"'{monitor_dir}' 디렉터리를 새로 만들었습니다.")

# 현재 디렉터리 내 파일 목록을 기록
initial_files = set(os.listdir(monitor_dir))

print(f"감시 시작, 현재 '{monitor_dir}' 폴더 내 파일 수: {len(initial_files)}개")

감시 시작, 현재 './monitor_directory' 폴더 내 파일 수: 0개


In [5]:
# 주요 확장자와 민감 정보 정규식 패턴 정의

# 주의할 확장자 리스트
suspicious_exts = ['.py', '.is', '.class']

# 주요 정보(주석, 이메일, SQL문) 탐지용 정규 표현식
comment_pattern = re.compile(r'#.*')  # 한 줄 주석 (파이썬 스타일)
email_pattern = re.compile(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')
sql_pattern = re.compile(r'\b(SELECT|INSERT|UPDATE|DELETE|DROP|ALTER|CREATE)\b', re.IGNORECASE)

In [ ]:
# 새로운 파일 감지 및 보안 위협 탐지 함수

def detect_new_files_and_threats():
    # 현재 디렉터리 파일 목록
    current_files = set(os.listdir(monitor_dir))
    
    # 새로 생성된 파일은 initial_files에 없는 파일
    new_files = current_files - initial_files
    
    if new_files:
        print(f"\n새로운 파일 발견: {new_files}")
    else:
        print("\n새 파일 없음")
    
    for filename in new_files:
        # 파일 경로 전체
        filepath = os.path.join(monitor_dir, filename)
        
        # 확장자 체크
        _, ext = os.path.splitext(filename)
        if ext in suspicious_exts:
            print(f"[주의] 주의 확장자 파일 발견: {filename}")
        
        # 파일 내용을 열어 민감 정보 탐지
        try:
            with open(filepath, "r", encoding="utf-8") as f:
                content = f.read()
                
                # 주석 탐지
                comments = comment_pattern.findall(content)
                if comments:
                    print(f"  -> 주석 발견 ({len(comments)}개)")
                
                # 이메일 탐지
                emails = email_pattern.findall(content)
                if emails:
                    print(f"  -> 이메일 주소 발견: {set(emails)}")
                
                # SQL문 탐지
                sqls = sql_pattern.findall(content)
                if sqls:
                    print(f"  -> SQL 키워드 발견: {set(sqls)}")
                    
        except Exception as e:
            print(f"  [오류] 파일 열기 실패: {filename} ({e})")

In [ ]:
# 모니터링 실행 루프 (예: 10초마다 3회 체크)

# 감시 반복 횟수와 간격 설정
check_count = 3
interval_sec = 10

for i in range(check_count):
    print(f"\n[{i+1}번째 체크 시작]")
    detect_new_files_and_threats()
    
    # 초기 파일 목록을 업데이트 (새 파일을 계속 추적하기 위해)
    initial_files.update(os.listdir(monitor_dir))
    time.sleep(interval_sec)
    
print("\n모니터링 종료")


[1번째 체크 시작]

새 파일 없음

[2번째 체크 시작]

새 파일 없음

[3번째 체크 시작]

새 파일 없음

모니터링 종료
